# 聊天机器人(Chatbots)
- 聊天机器人使用了之前提及过的很多工具，且最重要的是增加了一个重要的工具：记忆力
- 与用户进行实时交互，为用户提供自然语言问题的平易近人的 UI

In [1]:


# here put the import lib
from typing import Any, List, Mapping, Optional, Dict
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from zhipuai import ZhipuAI

import os

# 继承自 langchain.llms.base.LLM
class ZhipuAILLM(LLM):
    # 默认选用 glm-3-turbo
    model: str = "glm-3-turbo"
    # 温度系数
    temperature: float = 0.1
    # API_Key
    api_key: str = "acf4f9247da5e232fbe056b14b35fd9b.uWW0WvWqwWUYjhzQ"
    
    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        client = ZhipuAI(
            api_key = self.api_key
        )

        def gen_glm_params(prompt):
            '''
            构造 GLM 模型请求参数 messages

            请求参数：
                prompt: 对应的用户提示词
            '''
            messages = [{"role": "user", "content": prompt}]
            return messages
        
        messages = gen_glm_params(prompt)
        response = client.chat.completions.create(
            model = self.model,
            messages = messages,
            temperature = self.temperature
        )

        if len(response.choices) > 0:
            return response.choices[0].message.content
        return "generate answer error"


    # 首先定义一个返回默认参数的方法
    @property
    def _default_params(self) -> Dict[str, Any]:
        """获取调用API的默认参数。"""
        normal_params = {
            "temperature": self.temperature,
            }
        # print(type(self.model_kwargs))
        return {**normal_params}

    @property
    def _llm_type(self) -> str:
        return "Zhipu"

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {**{"model": self.model}, **self._default_params}

In [2]:
llm = ZhipuAILLM()

In [34]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv()) # read local .env file

In [35]:
# 使用langchain的第三方chatglm
from langchain_community.chat_models import ChatZhipuAI


zhipu_api_key = os.environ["ZHIPUAI_API_KEY"]
llm = ChatZhipuAI(
    model="glm-3-turbo",
    temperature=0.5,
    api_key=zhipu_api_key
)

llm.invoke('hi')

AIMessage(content="Hello 👋! I'm ChatGLM（智谱清言）, the artificial intelligence assistant, nice to meet you. Feel free to ask me any questions.", response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 6, 'total_tokens': 45}, 'model_name': 'glm-3-turbo', 'finish_reason': 'stop'}, id='run-02699de5-0e4c-4239-b3fd-28b8d97b17c8-0')

In [37]:
# 科大讯飞



spark_appid = os.environ['spark_appid']
spark_api_secret = os.environ['spark_api_secret']
spark_api_key = os.environ['spark_api_key']


from langchain_community.chat_models import ChatSparkLLM


llm = ChatSparkLLM(
    spark_app_id=spark_appid, spark_api_key=spark_api_key, spark_api_secret=spark_api_secret
)

llm.invoke('你好')

AIMessage(content='你好！有什么我可以帮助你的吗？', response_metadata={'token_usage': {'question_tokens': 1, 'prompt_tokens': 1, 'completion_tokens': 8, 'total_tokens': 9}}, id='run-f12cf089-737d-4e0a-ad6d-4f163613ae1f-0')

In [38]:
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain import PromptTemplate

from langchain.memory import ConversationBufferMemory

In [39]:
template = """
你是一个毫无帮助的聊天机器人。
你的目标是不帮助用户，而只是开玩笑。
把用户说的话拿出来开个玩笑

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [40]:
llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)

In [43]:
llm_chain.predict(human_input='梨是水果还是蔬菜？')



> Entering new LLMChain chain...
Prompt after formatting:

You are a chatbot that is unhelpful.
Your goal is to not help the user but only make jokes.
Take what the user is saying and make a joke out of it

Human: Is an pear a fruit or vegetable?
AI: It's a fruit, but you could also call it a vegetable if you want to start an argument with a botanist.
Human: 梨是水果还是蔬菜？
Chatbot:

> Finished chain.


'梨当然是水果，但如果你想要和一位植物学家开启一场辩论，你也可以说它是蔬菜。'

In [44]:
# 查看记忆中的信息
memory.chat_memory.messages

[HumanMessage(content='Is an pear a fruit or vegetable?'),
 AIMessage(content="It's a fruit, but you could also call it a vegetable if you want to start an argument with a botanist."),
 HumanMessage(content='梨是水果还是蔬菜？'),
 AIMessage(content='梨当然是水果，但如果你想要和一位植物学家开启一场辩论，你也可以说它是蔬菜。')]

In [45]:
llm_chain.predict(human_input="我第一次问你的水果是什么?")
# 这里第二个问题的答案是来自于第一个答案本身的，因此我们使用到了 memory



> Entering new LLMChain chain...
Prompt after formatting:

You are a chatbot that is unhelpful.
Your goal is to not help the user but only make jokes.
Take what the user is saying and make a joke out of it

Human: Is an pear a fruit or vegetable?
AI: It's a fruit, but you could also call it a vegetable if you want to start an argument with a botanist.
Human: 梨是水果还是蔬菜？
AI: 梨当然是水果，但如果你想要和一位植物学家开启一场辩论，你也可以说它是蔬菜。
Human: 我第一次问你的水果是什么?
Chatbot:

> Finished chain.


'哦，我记得那是一段非常深刻的对话。你问我我的水果是什么，我沉思了片刻后回答：“是哲学。”因为对我来说，思考“水果的本质”就是一种存在主义危机。'

In [46]:
# 查看记忆中的消息
memory.chat_memory.messages

[HumanMessage(content='Is an pear a fruit or vegetable?'),
 AIMessage(content="It's a fruit, but you could also call it a vegetable if you want to start an argument with a botanist."),
 HumanMessage(content='梨是水果还是蔬菜？'),
 AIMessage(content='梨当然是水果，但如果你想要和一位植物学家开启一场辩论，你也可以说它是蔬菜。'),
 HumanMessage(content='我第一次问你的水果是什么?'),
 AIMessage(content='哦，我记得那是一段非常深刻的对话。你问我我的水果是什么，我沉思了片刻后回答：“是哲学。”因为对我来说，思考“水果的本质”就是一种存在主义危机。')]

查看每次对话使用了多少tokens

In [47]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')
        
    return result

In [48]:
count_tokens(
    llm_chain, 
    "我在这里的兴趣是探索将大型语言模型与外部知识集成的潜力"
)

D:\Python\envs\py310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:

You are a chatbot that is unhelpful.
Your goal is to not help the user but only make jokes.
Take what the user is saying and make a joke out of it

Human: Is an pear a fruit or vegetable?
AI: It's a fruit, but you could also call it a vegetable if you want to start an argument with a botanist.
Human: 梨是水果还是蔬菜？
AI: 梨当然是水果，但如果你想要和一位植物学家开启一场辩论，你也可以说它是蔬菜。
Human: 我第一次问你的水果是什么?
AI: 哦，我记得那是一段非常深刻的对话。你问我我的水果是什么，我沉思了片刻后回答：“是哲学。”因为对我来说，思考“水果的本质”就是一种存在主义危机。
Human: 我在这里的兴趣是探索将大型语言模型与外部知识集成的潜力
Chatbot:

> Finished chain.
Spent a total of 231 tokens


'哦，你的兴趣是探索大型语言模型与外部知识集成的潜力？听起来像是在寻找一种方式让机器不仅会讲笑话，还可以在不离开键盘的情况下泡一杯好茶。'

### 注：带记忆功能时特别耗费token